# CS4049 Assessment 2:

This assessment requires the use of a Taxi environment to train a model, using OpenAI. 

In [3]:
import gymnasium as gym #For the environment.
import tensorflow as tf
import keras 


In [6]:
env = gym.make('Taxi-v3', render_mode="human")
print(env)
env.reset()
action = env.action_space.sample()
observation, reward, terminated, truncated, info = env.step(action)

print(action)



<TimeLimit<OrderEnforcing<PassiveEnvChecker<TaxiEnv<Taxi-v3>>>>>
1
